In [ ]:
import pandas as pd
import numpy as np
!pip install -U gensim
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora
from gensim.models import Phrases
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
from sklearn.manifold import TSNE
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import spacy

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

In [ ]:
!pip install pyLDAvis==2.1.2

import pyLDAvis
import pyLDAvis.gensim

In [ ]:
fin_ind = pd.read_csv('fin_with_ind.csv')
id_ind = fin_ind[['id', 'Industry Sector', 'Primary Industry Classification']]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
corpus_root = '/content/drive/MyDrive/10-k'
corpus = PlaintextCorpusReader(corpus_root, '.*')

In [ ]:
rows = []
for filename in corpus.fileids():
  rows.append((filename, corpus.raw(filename)))

df = pd.DataFrame(rows, columns=['name', 'text'])

In [ ]:
def find_id(s):
  return int(''.join(x for x in s if x.isdigit()))

df['id'] = df['name'].apply(find_id)

In [ ]:
data = df.merge(id_ind, on='id')

In [ ]:
data.groupby('Industry Sector').count()

In [ ]:
#df by sectors
industrials_df = data[data['Industry Sector'] == 'Industrials']
condiscret_df = data[data['Industry Sector'] == 'Consumer Discretionary']
it_df = data[data['Industry Sector'] == 'Information Technology']
energy_df = data[data['Industry Sector'] == 'Energy']
healthcare_df = data[data['Industry Sector'] == 'Health Care']
comservice_df = data[data['Industry Sector'] == 'Communication Services']
constaple_df = data[data['Industry Sector'] == 'Consumer Staples']
materials_df = data[data['Industry Sector'] == 'Materials']
realestate_df = data[data['Industry Sector'] == 'Real Estate']
utilities_df = data[data['Industry Sector'] == 'Utilities']

In [ ]:
stop_words = stopwords.words('english')

def get_pos(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {'J':wordnet.ADJ,
              'N':wordnet.NOUN,
              'V':wordnet.VERB,
              'R':wordnet.ADV}
  return tag_dict.get(tag, wordnet.NOUN)

def token_lemmatize(token):
  lemmatizer = WordNetLemmatizer()
  return lemmatizer.lemmatize(token, get_pos(token))

In [ ]:
#noise from Industrials
utilities_corpus = []
for i in range(len(utilities_df)):  
  text = utilities_df.iloc[i]['text']
  text = simple_preprocess(text)
  lemmatized = [token_lemmatize(word) for word in text if word not in stop_words]
  result = [word for word in lemmatized if word not in stop_words]
  utilities_corpus.append(result)

#bigram_phrases = gensim.models.Phrases(constaple_corpus, min_count=5, threshold=50)
#bigram = gensim.models.phrases.Phraser(bigram_phrases)
#bigram_corpus = [bigram[doc] for doc in constaple_corpus]

id2word = corpora.Dictionary(utilities_corpus)
train_corpus = [id2word.doc2bow(text) for text in utilities_corpus]


In [ ]:
tfidf = TfidfModel(train_corpus, id2word=id2word)

low_value = 0.04
drop_words = [] #dropped words
for i in range(len(train_corpus)):
  bow = train_corpus[i]
  low_value_words = []
  tfidf_ids = [id for id, value in tfidf[bow]]
  bow_ids = [id for id, value in bow]
  low_value_words = [id for id, value in tfidf[bow] if value < low_value]
  words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] #words with tf-idf score 0 will be missing
  drops = low_value_words + words_missing_in_tfidf
  for item in drops:
    drop_words.append(id2word[item])
  new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
  train_corpus[i] = new_bow

In [ ]:
noise = pd.DataFrame(drop_words)[0].unique()
len(noise)

4334

In [ ]:
' '.join(noise)

'ability able acceptable accepted access accordance accordingly account accounting accrue accumulate acequia achieve acquire acquisition acre act action activate actively actual actuarially addition additional additionally address adequate adjacent adjust adjustment adjusts administer administrative adopt adoption advance adversely advice affect affected affirmative agency aggregate agree aid aligns alliance allocation allow allows also alternative among amortization amortize amount analysis announce annual annually antenna anticipate anticipates antonio applicable application applies apply approach appropriate approval approve approximate approximately april area arise arizona arrangement asc aspect asphalt assess assignment associate assume assumption attract attributable audit austin authority authorization authorize availability available average award back bank base basin basis bear become becomes begin believe beneficiary benefit beyond bi bill billing blanco board bond borrower 

In [ ]:
ldaModel = gensim.models.ldamodel.LdaModel(corpus=train_corpus, 
                                           id2word=id2word, 
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           update_every=1,
                                           passes=10)

In [ ]:
doc_len = []
for i in range(len(train_corpus)):
  doc_len.append(len(train_corpus[i]))
np.mean(doc_len), np.median(doc_len), np.min(doc_len), np.max(doc_len)

(30.8125, 25.5, 11, 80)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(topic_model=ldaModel, corpus=train_corpus, dictionary=id2word, mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.314920  0.099589       1        1  19.498169
4     -0.007285  0.371821       2        1  17.026008
3     -0.342880 -0.097084       3        1  16.220381
9     -0.251446  0.219148       4        1  12.475240
0      0.259625 -0.286660       5        1  10.458131
8      0.211932  0.168552       6        1   7.475553
6     -0.085105 -0.359472       7        1   6.934364
1     -0.141426 -0.000665       8        1   4.542140
2      0.038070  0.027476       9        1   4.480754
5      0.003595 -0.142705      10        1   0.889261, topic_info=               Term        Freq       Total Category  logprob  loglift
1975          idaho  524.000000  524.000000  Default  30.0000  30.0000
734           power  642.000000  642.000000  Default  29.0000  29.0000
4463            pnm  287.000000  287.000000  Default  28.0000  28.0000
1089          water  481.000000  481.000000  Default  27.0000  27.0000
1337            gas  414.000000  414.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
2929  subcontractor    0.263559    0.954453  Topic10  -6.3075   3.4357
2935      worldwide    0.263547    0.954488  Topic10  -6.3075   3.4356
266        delaware    0.981086   12.773171  Topic10  -4.9931   2.1561
255        december    4.138063  270.995238  Topic10  -3.5538   0.5407
230     corporation    0.983097   46.595586  Topic10  -4.9910   0.8640

[445 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2938      7  0.957770  abandonment
19       10  0.833793     actively
2893     10  0.834264     advisory
3263      1  0.984982          ael
1695      6  0.639956        afudc
...     ...       ...          ...
3256      9  0.908019        white
2806      6  0.980810          wna
3633      8  0.932434         wolf
1110      9  0.373971         york
1110     10  0.373971         york

[327 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 4, 10, 1, 9, 7, 2, 3, 6])

In [ ]:
len(vis.topic_info['Term'].unique())

320

In [ ]:
sw = ' '.join(vis.topic_info['Term'].unique().tolist())
sw